In [1]:
# https://www.geeksforgeeks.org/scrape-linkedin-using-selenium-and-beautiful-soup-in-python/

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

## Logging in to LinkedIn

In [3]:
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Joyce\AppData\Local\Temp\ipykernel_10424\2930654737.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
# waiting for the page to load
time.sleep(2)

In [5]:
# entering username
username = driver.find_element(By.ID, "username")
# Enter Your Email Address
username.send_keys("sh2429@cornell.edu")

In [6]:
# entering password
pword = driver.find_element(By.ID, "password")
# Enter Your Password
pword.send_keys("qzwx.1506")

In [7]:
# Clicking on the log in button
# Format (syntax) of writing XPath -->
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

## Extracting Profile Data

In [8]:
# # Opening Joyce's Profile
# # paste the URL of Joyce's profile here
# profile_url = "https://www.linkedin.com/in/shuyan-huang-cs/"
# # this will open the link
# driver.get(profile_url)

In [9]:
# Now, we need to scroll to the bottom. Here is the code to do that:
start = time.time()

# will be used in the while loop
initialScroll = 0
finalScroll = 1000

while True:
    driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
    # this command scrolls the window starting from
    # the pixel value stored in the initialScroll
    # variable to the pixel value stored at the
    # finalScroll variable
    initialScroll = finalScroll
    finalScroll += 1000
 
    # we will stop the script for 3 seconds so that the data can load
    time.sleep(3)
    # You can change it as per your needs and internet speed
 
    end = time.time()
 
    # We will scroll for 20 seconds.
    # You can change it as per your needs and internet speed
    if round(end - start) > 20:
        break

In [10]:
# src = driver.page_source
 
# # Now using beautiful soup
# soup = BeautifulSoup(src, 'lxml')

### Extract Introduction

In [11]:
# # Extracting the HTML of the complete introduction box
# # that contains the name, company name, and the location
# intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

# print(intro)

In [12]:
# # In case of an error, try changing the tags used here.
 
# name_loc = intro.find("h1")
 
# # Extracting the Name
# name = name_loc.get_text().strip()
# # strip() is used to remove any extra blank spaces
 
# works_at_loc = intro.find("div", {'class': 'text-body-medium'})

# # this gives us the HTML of the tag in which the Company Name is present
# # Extracting the Company Name
# works_at = works_at_loc.get_text().strip()

# print("Name -->", name,
#       "\nWorks At -->", works_at)

## Extracting Job Search Data

In [13]:
# Opening job's page
# paste the URL of job's page here
job_url = "https://www.linkedin.com/jobs/"
# this will open the link
driver.get(job_url)
time.sleep(2)

In [14]:
# entering search keywords
search = driver.find_element(By.XPATH, "//input[starts-with(@id, 'jobs-search-box-keyword-id-ember')]")
search.send_keys("software engineer")
time.sleep(2)
search.send_keys(Keys.RETURN)
# <input id="jobs-search-box-keyword-id-ember26" class="jobs-search-box__text-input jobs-search-box__keyboard-text-input" autocomplete="chrome-off" spellcheck="false" role="combobox" aria-autocomplete="list" aria-label="Search by title, skill, or company" aria-activedescendant="" aria-expanded="false" type="text">

In [15]:
# scroll
time.sleep(2)
left_panel = driver.find_element(By.CLASS_NAME, "jobs-search-results-list")
time.sleep(1)
verical_ordinate = 100
for i in range(0, 50):
   driver.execute_script("arguments[0].scrollTop = arguments[1]", left_panel, verical_ordinate)
   verical_ordinate += 100
   time.sleep(1)

In [16]:
job_src = driver.page_source
soup = BeautifulSoup(job_src, 'lxml')

In [17]:
jobs_html = soup.find_all('a', {'class': 'job-card-list__title'})

job_titles = []
 
for title in jobs_html:
    job_titles.append(title.text.strip())

print(job_titles)

['Python Developer', 'Software Engineer, Full Stack', 'NESD BMC Remedy/Helix Developer Sr.', 'Software Engineer Staff - TS/SCI w/ Poly (*Up to $25K Sign-On Bonus*)', 'Machine Learning Research Engineer II', 'Backend Engineer - Data Modelling', '2023 Internship - RCP Research Scientist', 'Backend Developer', 'Web Developer', 'Front-End Engineer (Vue.js) - Remote', 'Chapter Next Returnship- Embedded Software Engineer', 'Simulation and Machine Learning Engineer, Ad Platforms', 'Software Engineer Intern', 'Senior Software Engineer', 'Backend Engineer', 'Back End Developer', 'Software Engineer - Early Career', 'Staff - NLP Machine Learning Engineer', 'Quantitative Developer', 'Python Backend Engineer', 'Backend Engineer - Contract', 'Junior Software Engineer', 'Machine Learning Engineer - All Levels (REMOTE)']


In [18]:
company_name_html = soup.find_all(
  'a', {'class': 'job-card-container__company-name'})
company_names = []

for name in company_name_html:
    company_names.append(name.text.strip())

print(company_names)

['Collabera', 'Capital One', 'General Dynamics Information Technology', 'Lockheed Martin', 'R-DEX Systems, Inc.', 'Clevertech', "Landry's", 'Symphony Talent Middleware Test Company', 'Dice', 'runZero', 'Lockheed Martin', 'Apple', 'Domo, Inc.', 'TeachFX', 'CloserIQ', 'Pattern Bioscience', 'Lockheed Martin', 'Jasper', 'Arrowstreet Capital, Limited Partnership', 'Motion Recruitment', 'PAX', 'Wiley Edge', 'GEICO']
